In [6]:
import emcee

In [7]:
import numpy as np
import scipy.stats
import matplotlib.pyplot as plt
import time

class MetropolisHastings:
    def __init__(self, target_distr, initial_state=None, seed=None):
        """
        Initialize the Metropolis-Hastings sampler.
        
        Parameters:
        - target_distr: function
            The target distribution from which we want to sample.
        - initial_state: array-like, optional
            The initial state of the Markov chain. Default is a zero vector.
        - seed: int, optional
            Random seed for reproducibility. Default is None.
        """
        self.target_distr = target_distr
        self.dim = len(initial_state) if initial_state is not None else 1
        self.initial_state = np.array(initial_state) if initial_state is not None else np.zeros(self.dim)
        self.seed = seed
        self.samples = []
        self.acceptance_rate = 0.0
        self.mean = None
        self.covariance = None
        self.time = 0.0
        
        if seed is not None:
            np.random.seed(seed)
    
    def sample(self, n_samples):
        """
        Generate samples using the Metropolis-Hastings algorithm.
        
        Parameters:
        - n_samples: int
            The number of samples to generate.
        
        Returns:
        - samples: np.array
            Array of generated samples.
        """
        start_time = time.time()
        
        proposal_distr = scipy.stats.multivariate_normal
        # Random covariance matrix for the proposal distribution
        A = np.random.rand(self.dim, self.dim)
        cov = np.dot(A, A.T)
        
        x_t = self.initial_state
        samples = []
        accept_count = 0
        
        while len(samples) < n_samples:
            x_prime = proposal_distr.rvs(mean=x_t, cov=cov)
            acceptance_ratio = (self.target_distr(x_prime) * proposal_distr.pdf(x_t, mean=x_prime, cov=cov)) / \
                               (self.target_distr(x_t) * proposal_distr.pdf(x_prime, mean=x_t, cov=cov))
            
            acceptance_ratio = min(1, acceptance_ratio)
            if np.random.uniform(0, 1) < acceptance_ratio:
                x_t = x_prime
                samples.append(x_t)
                accept_count += 1
            else:
                samples.append(x_t)
        
        self.samples = np.array(samples)
        self.acceptance_rate = accept_count / n_samples
        self.mean = np.mean(self.samples, axis=0)
        self.covariance = np.cov(self.samples.T)
        
        self.time = time.time() - start_time
        
        return self.samples

In [8]:
# Define the true mean and covariance for the 5-dimensional Gaussian
true_mean = np.zeros(5)
true_cov = np.eye(5)  # Identity matrix as covariance

# Define the target distribution function
def target_distr(x):
    return scipy.stats.multivariate_normal(mean=true_mean, cov=true_cov).pdf(x)

# Initialize the Metropolis-Hastings sampler
mh_sampler = MetropolisHastings(target_distr=target_distr, initial_state=np.zeros(5), seed=42)
mh_samples = mh_sampler.sample(10000)

# Print the mean and covariance estimated by Metropolis-Hastings
print("Metropolis-Hastings Mean:\n", mh_sampler.mean)
print("Metropolis-Hastings Covariance:\n", mh_sampler.covariance)

# Use the emcee package for comparison
nwalkers = 10
ndim = 5
nsteps = 10000

# Define the log probability function for emcee
def log_prob(x):
    return np.log(target_distr(x))

# Initialize the walkers
initial_positions = np.random.rand(nwalkers, ndim)

# Set up the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)
sampler.run_mcmc(initial_positions, nsteps)

# Get the samples from the emcee sampler
emcee_samples = sampler.get_chain(flat=True)

# Print the mean and covariance estimated by emcee
print("emcee Mean:\n", np.mean(emcee_samples, axis=0))
print("emcee Covariance:\n", np.cov(emcee_samples, rowvar=False))

# Print the true mean and covariance
print("True Mean:\n", true_mean)
print("True Covariance:\n", true_cov)

Metropolis-Hastings Mean:
 [-0.03484152 -0.05886685  0.11731985 -0.05781232  0.15072478]
Metropolis-Hastings Covariance:
 [[ 0.92726408 -0.13618451  0.05337548  0.32141935 -0.03193578]
 [-0.13618451  0.88967914  0.00283318  0.27792316 -0.01982301]
 [ 0.05337548  0.00283318  1.00630077 -0.08258819  0.08236611]
 [ 0.32141935  0.27792316 -0.08258819  0.2458768  -0.07511317]
 [-0.03193578 -0.01982301  0.08236611 -0.07511317  1.15429098]]
emcee Mean:
 [-0.0529077  -0.03640212  0.0107693   0.03261933 -0.01776596]
emcee Covariance:
 [[ 1.03702416  0.03907741  0.01405305 -0.00651264 -0.00139316]
 [ 0.03907741  0.98517786  0.00571647 -0.0120379   0.0299465 ]
 [ 0.01405305  0.00571647  0.98785319 -0.00728946  0.02012409]
 [-0.00651264 -0.0120379  -0.00728946  0.99949329 -0.01884306]
 [-0.00139316  0.0299465   0.02012409 -0.01884306  1.01387543]]
True Mean:
 [0. 0. 0. 0. 0.]
True Covariance:
 [[1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


In [9]:
mh_sampler.time

2.4082398414611816

In [10]:
# Define the true mean and covariance for the 11-dimensional Gaussian
true_mean = np.zeros(11)
true_cov = np.eye(11)  # Identity matrix as covariance

# Define the target distribution function
def target_distr(x):
    return scipy.stats.multivariate_normal(mean=true_mean, cov=true_cov).pdf(x)

# Initialize the Metropolis-Hastings sampler
mh_sampler = MetropolisHastings(target_distr=target_distr, initial_state=np.zeros(11), seed=42)
mh_samples = mh_sampler.sample(10000)

# Print the mean and covariance estimated by Metropolis-Hastings
print("Metropolis-Hastings Mean:\n", mh_sampler.mean)
print("Metropolis-Hastings Covariance:\n", mh_sampler.covariance)

# Use the emcee package for comparison
nwalkers = 22  # Usually 2 * ndim
ndim = 11
nsteps = 10000

# Define the log probability function for emcee
def log_prob(x):
    return np.log(target_distr(x))

# Initialize the walkers
initial_positions = np.random.rand(nwalkers, ndim)

# Set up the sampler
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_prob)
sampler.run_mcmc(initial_positions, nsteps)

# Get the samples from the emcee sampler
emcee_samples = sampler.get_chain(flat=True)

# Print the mean and covariance estimated by emcee
print("emcee Mean:\n", np.mean(emcee_samples, axis=0))
print("emcee Covariance:\n", np.cov(emcee_samples, rowvar=False))

# Print the true mean and covariance
print("True Mean:\n", true_mean)
print("True Covariance:\n", true_cov)

Metropolis-Hastings Mean:
 [-0.1260418   0.21491756 -0.34513941 -0.09328139 -0.0188139  -0.14479388
  0.36698198 -0.49225039  0.06852211 -0.17544672  0.48904296]
Metropolis-Hastings Covariance:
 [[ 1.01849373  0.09656926  0.08844295 -0.06479121  0.11243986 -0.04546604
   0.02781775 -0.08190013 -0.1695272   0.01395003  0.05536088]
 [ 0.09656926  0.60082284 -0.07000666  0.06367828  0.01551438  0.03524322
  -0.18484585  0.09820924  0.19802685 -0.04709906 -0.10677527]
 [ 0.08844295 -0.07000666  0.98493256 -0.00418638 -0.02380501 -0.1302352
   0.03325687 -0.04507815  0.1359716  -0.06257436  0.16963536]
 [-0.06479121  0.06367828 -0.00418638  0.87316515  0.02816196  0.04667738
   0.13496196 -0.07887649 -0.09412446  0.0127171   0.11297371]
 [ 0.11243986  0.01551438 -0.02380501  0.02816196  0.86165251 -0.11404901
   0.08566747 -0.20944717 -0.12914816  0.02327722  0.12610025]
 [-0.04546604  0.03524322 -0.1302352   0.04667738 -0.11404901  0.94088031
   0.04717639 -0.01459212  0.07933502 -0.020178

In [11]:
mh_sampler.time

2.973867893218994